In [78]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model

In [79]:
import warnings
warnings.filterwarnings("ignore")

In [80]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [81]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [82]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [83]:
answers = {}

In [84]:
# Some data structures that will be useful

In [85]:
allRatings = []
userRatings = defaultdict(list)

for user,book,r in readCSV("assignment1/train_Interactions.csv.gz"):
  r = int(r)
  allRatings.append(r)
  userRatings[user].append(r)

In [86]:
len(allRatings)

200000

In [87]:
allInteractions = [(user, book, int(r)) for user, book, r in readCSV("assignment1/train_Interactions.csv.gz")]

In [88]:
globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
  userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

In [89]:
ratingsTrain = allInteractions[:190000]  # Use tuples of (user, book, rating)
ratingsValid = allInteractions[190000:]  # Validation set with tuples of (user, book, rating)

ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u, b, r in ratingsTrain:  # Unpack the tuples properly
    ratingsPerUser[u].append((b, r))  # Append book-rating pairs
    ratingsPerItem[b].append((u, r))  # Append user-rating pairs

In [90]:
allRatings[0]

3

### Tasks (Read prediction)
Since we don’t have access to the test labels, we’ll need to simulate validation/test sets of our own.
So, let’s split the training data (‘train Interactions.csv.gz’) as follows:
(1) Reviews 1-190,000 for training
(2) Reviews 190,001-200,000 for validation
(3) Upload only when you have a good model on the validation set

In [91]:
##################################################
# Read prediction                                #
##################################################

In [92]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("assignment1/train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [93]:
# Create sets of users who have read each book
usersPerItem = defaultdict(set)
for u, b, _ in ratingsTrain:
    usersPerItem[b].add(u)

# Create a dictionary with all books read by each user in the training data
itemsPerUser = defaultdict(set)
for u, b, _ in ratingsTrain:
    itemsPerUser[u].add(b)

### Question 1
Although we have built a validation set, it only consists of positive samples. For this task we also need
examples of user/item pairs that weren’t read. For each (user,book) entry in the validation set, sample a
negative entry by randomly choosing a book that user hasn’t read. Evaluate the performance (accuracy)
of the baseline model on the validation set you have built.

In [94]:
# Create validation set with negative samples
validation_with_negatives = []
for u, b, r in ratingsValid:
    # Positive sample (user has read this book)
    validation_with_negatives.append((u, b, 1))  
    
    # Negative sample (user hasn't read this book)
    while True:
        negative_book = random.choice(list(bookCount.keys()))
        if negative_book not in itemsPerUser[u]:
            validation_with_negatives.append((u, negative_book, 0))
            break
# Baseline prediction based on popularity
def baseline_predict(user, book):
    return 1 if book in return1 else 0

# Calculate accuracy as a float value for baseline model on validation set
correct_predictions = 0
total_predictions = len(validation_with_negatives)

for u, b, actual in validation_with_negatives:
    prediction = baseline_predict(u, b)
    if prediction == actual:
        correct_predictions += 1

acc1 = correct_predictions / total_predictions

In [95]:
answers['Q1'] = acc1

In [96]:
acc1

0.71285

In [97]:
assertFloat(answers['Q1'])

### Question 2
The existing ‘read prediction’ baseline just returns True if the item in question is ‘popular,’ using a
threshold based on those books which account for 50% of all interactions (totalRead/2). Assuming that
the ‘non-read’ test examples are a random sample of user-book pairs, this threshold may not be the best
one. See if you can find a better threshold (or otherwise modify the threshold strategy); report the new threshold and its performance of your improved model on your validation set


In [98]:
# Possible thresholds (percentages of total reads that books need to reach)
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
best_threshold = 0.5  # Start with the initial threshold
best_accuracy = 0

# Iterate through each threshold and evaluate performance
for threshold in thresholds:
    # Calculate the interaction count required for this threshold
    count_threshold = totalRead * threshold

    # Update return1 to include books that meet the threshold
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > count_threshold:
            break

    # Calculate accuracy with this threshold
    correct_predictions = 0
    total_predictions = len(validation_with_negatives)

    for u, b, actual in validation_with_negatives:
        prediction = baseline_predict(u, b)
        if prediction == actual:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions

    # Update best threshold and accuracy if this threshold is better
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold


In [99]:
answers['Q2'] = [best_threshold, best_accuracy]

In [100]:
[best_threshold, best_accuracy]

[0.7, 0.7535]

In [101]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

### Question 3
A stronger baseline (This baseline is not always stronger, depending on the dataset.) 
than the one provided might make use of the Jaccard similarity (or another similarity
metric). Given a pair (u, b) in the validation set, consider all training items b′that user u has read. 
For each, compute the Jaccard similarity between b and b′, 
i.e., users (in the training set) who have read b and users who have read b′. 
Predict as ‘read’ if the maximum of these Jaccard similarities exceeds a threshold 
(you may choose the threshold that works best). Report the performance on your validation set.

In [102]:
import math

# Cosine similarity calculation based on co-occurrence
def cosine_similarity(set1, set2):
    intersection_size = len(set1 & set2)
    norm1 = math.sqrt(len(set1))
    norm2 = math.sqrt(len(set2))
    return intersection_size / (norm1 * norm2) if norm1 > 0 and norm2 > 0 else 0

# Combined prediction function using popularity and similarity
def combined_predict(user, book, popularity_threshold, similarity_threshold):
    # Step 1: Check if the book is popular enough
    if book in return1_popular_books:  # Use popular books set
        return 1  # Predict "read" if the book is popular

    # Step 2: Check similarity with books the user has read
    user_books = itemsPerUser.get(user, set())
    max_similarity = 0
    
    for b_prime in user_books:
        similarity = cosine_similarity(usersPerItem.get(book, set()), usersPerItem.get(b_prime, set()))
        max_similarity = max(max_similarity, similarity)
        
    # Predict "read" if the maximum similarity exceeds the similarity threshold
    return 1 if max_similarity >= similarity_threshold else 0

# Define thresholds to test for both popularity and similarity
popularity_thresholds = [0.1, 0.2, 0.3, 0.4, 0.5]  # Test various percentages
similarity_thresholds = [0.05, 0.1, 0.15, 0.2, 0.25]  # Finer increments for similarity

best_accuracy = 0
best_popularity_threshold = 0
best_similarity_threshold = 0

# Evaluate performance by iterating over popularity and similarity thresholds
for popularity_threshold in popularity_thresholds:
    # Determine popular books based on the current popularity threshold
    count_threshold = totalRead * popularity_threshold
    return1_popular_books = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1_popular_books.add(i)
        if count > count_threshold:
            break
    
    for similarity_threshold in similarity_thresholds:
        correct_predictions = 0
        total_predictions = len(validation_with_negatives)
        
        # Evaluate each validation pair using the combined model
        for u, b, actual in validation_with_negatives:
            prediction = combined_predict(u, b, popularity_threshold, similarity_threshold)
            if prediction == actual:
                correct_predictions += 1
        
        # Calculate accuracy for the current threshold combination
        accuracy = correct_predictions / total_predictions
        
        # Update best thresholds if current combination performs better
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_popularity_threshold = popularity_threshold
            best_similarity_threshold = similarity_threshold

# Output the best accuracy and threshold combination
acc3 = best_accuracy
print(f"Best Popularity Threshold: {best_popularity_threshold}, Best Similarity Threshold: {best_similarity_threshold}, Best Accuracy: {best_accuracy}")

Best Popularity Threshold: 0.5, Best Similarity Threshold: 0.1, Best Accuracy: 0.71305


In [103]:
acc3 = best_accuracy 
acc3

0.71305

### Question 4
Improve the above predictor by incorporating both a Jaccard-based threshold and a popularity based
threshold. Report the performance on your validation set.

In [104]:
# Define Jaccard similarity function
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

# Step 1: Find the best Jaccard threshold first (from Question 3)
best_jaccard_threshold = 0.3  # Assume this was optimal from Q3

# Step 2: Optimize popularity threshold only, given the best Jaccard threshold
best_combined_accuracy = 0
best_popularity_threshold = 0.5  # Starting point for popularity threshold

# Smaller set of popularity thresholds for quicker testing
popularity_thresholds = [0.3, 0.5, 0.7]  # Adjust as needed

# Memoization dictionary for Jaccard similarities
jaccard_cache = {}

for p_threshold in popularity_thresholds:
    # Calculate popularity set for the given threshold
    count_threshold = totalRead * p_threshold
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > count_threshold:
            break

    correct_predictions = 0
    total_predictions = len(validation_with_negatives)

    # Predict with combined Jaccard and popularity threshold
    for u, b, actual in validation_with_negatives:
        # Check Jaccard-based prediction
        if u not in itemsPerUser or not itemsPerUser[u]:
            jaccard_prediction = 0  # No history for user
        else:
            max_similarity = max(
                jaccard_cache.setdefault(
                    (b, b_prime), jaccard_similarity(usersPerItem[b], usersPerItem[b_prime])
                )
                for b_prime in itemsPerUser[u]
            )
            jaccard_prediction = 1 if max_similarity > best_jaccard_threshold else 0

        # Check popularity-based prediction
        popularity_prediction = 1 if b in return1 else 0

        # Combined prediction (logical OR)
        prediction = 1 if jaccard_prediction == 1 or popularity_prediction == 1 else 0

        # Compare to actual label
        if prediction == actual:
            correct_predictions += 1

    # Calculate accuracy
    accuracy = correct_predictions / total_predictions
    if accuracy > best_combined_accuracy:
        best_combined_accuracy = accuracy
        best_popularity_threshold = p_threshold

acc4 = best_combined_accuracy

In [105]:
acc4

0.7535

In [106]:
answers['Q3'] = acc3
answers['Q4'] = acc4

In [107]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

# Question 5
To run our model on the test set, we’ll have to use the files ‘pairs Read.csv’ to find the userID/bookID
pairs about which we have to make predictions. Using that data, run the above model and upload your
solution to the (Assignment 1) Gradescope. If you’ve already uploaded a better solution, that’s fine
too! Your answer should be the string “I confirm that I have uploaded an assignment submission to
gradescope”.

In [ ]:
predictions = open("predictions_Read.csv", 'w')
for l in open("assignment1/pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    # (etc.)

predictions.close()

In [109]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [110]:
assert type(answers['Q5']) == str

In [111]:
##################################################
# Rating prediction                              
# Let’s start by building our training/validation sets much as we did for the first task. This time building a
# validation set is more straightforward: you can simply use part of the data for validation, and do not need to
# randomly sample non-read users/books
##################################################

•	For Question 6, we are tasked with using a fixed regularization parameter \lambda = 1 and minimizing the Sum of Squared Errors (SSE), including a regularization term for user and item biases.
•	The goal here is to report the MSE on the validation set after training the model with \lambda = 1.

•	In Question 8, we are tuning \lambda by trying multiple values (e.g., [0.01, 0.1, 1, 10, 100]) and finding the one that yields the best performance on the validation set.
•	We then report the optimal \lambda, along with the MSE for that \lambda.

### Question 6
Fit a predictor of the form
rating(user, item) ≃ α + βuser + βitem,
by fitting the mean and the two bias terms as described in the lecture notes. Use a regularization
parameter of λ = 1. Report the MSE on the validation set.
For this question note carefully that the objective optimized should be the sum of squared errors plus
the regularizer (squared ℓ2 norm scaled by λ), i.e., it should not be the mean squared error. Although
using the MSE vs SSE is equivalent (up to a change in λ) it is important that your objective follows this
exact specification so that everyone’s solution is the same. 

In [112]:
from collections import defaultdict

# Initialize parameters for the model
alpha = 0  # Global bias
user_bias = defaultdict(float)  # β_user
item_bias = defaultdict(float)  # β_item
lambda_reg = 1  # Regularization parameter

# Derive unique user and book sets from ratingsTrain
train_user_ids = set(u for u, _, _ in ratingsTrain)
train_book_ids = set(b for _, b, _ in ratingsTrain)

# Objective function: Sum of Squared Errors (SSE) with regularization
def train_bias_model_fixed_lambda(user_ids, item_ids, ratings, iterations=10, learning_rate=0.01, lambda_reg=1):
    global alpha, user_bias, item_bias
    alpha = 1.0  # Initialize global bias
    user_bias = defaultdict(float)  # Reset user biases
    item_bias = defaultdict(float)  # Reset item biases

    for _ in range(iterations):
        for u, i, r in ratings:
            # Predicted rating based on current parameters
            pred = alpha + user_bias[u] + item_bias[i]
            
            # Calculate error for this prediction
            error = r - pred  # Target is 1 for "read" status
            
            # Update parameters with regularization for SSE
            alpha += learning_rate * error
            user_bias[u] += learning_rate * (error - lambda_reg * user_bias[u])
            item_bias[i] += learning_rate * (error - lambda_reg * item_bias[i])

# Train the model on the training data with λ = 1
train_bias_model_fixed_lambda(train_user_ids, train_book_ids, ratingsTrain, lambda_reg=lambda_reg)

# Calculate MSE on the validation set
def calculate_mse(validation_ratings):
    mse_total = 0
    count = 0
    for u, i, r in validation_ratings:
        # Predicted rating
        pred = alpha + user_bias[u] + item_bias[i]
        
        # Compute squared error
        mse_total += (r - pred) ** 2
        count += 1
    return mse_total / count if count > 0 else float('nan')

# Store the MSE for the validation set for Question 6
validMSE = calculate_mse(ratingsValid)

In [113]:
answers['Q6'] = validMSE

In [114]:
assertFloat(answers['Q6'])

In [115]:
validMSE

1.488958799100907

### Question 7
Report the user IDs that have the largest and smallest (i.e., largest negative) values of βu, along with
the beta values.

In [116]:
# Ensure that Question 6 has been run to compute user biases
# After training the bias model, extract user bias values for analysis

# Get the user bias values (from the model trained in Q6)
user_bias_values = [(user, beta) for user, beta in user_bias.items()]

# Find the user with the largest positive and largest negative biases
maxUser, maxBeta = max(user_bias_values, key=lambda x: x[1])  # User with highest positive bias
minUser, minBeta = min(user_bias_values, key=lambda x: x[1])  # User with largest negative bias

# Ensure user IDs are converted to strings and biases are floats
maxUser = str(maxUser)
minUser = str(minUser)
maxBeta = float(maxBeta)
minBeta = float(minBeta)

In [117]:
answers['Q7'] = [maxUser, minUser, maxBeta, minBeta]

In [118]:
assert [type(x) for x in answers['Q7']] == [str, str, float, float]

In [119]:
[maxUser, minUser, maxBeta, minBeta]

['u79275096', 'u88024921', 0.6925560635645533, -1.8178145604516576]

### Question 8
Find a better value of λ using your validation set. Report the value you chose, its MSE, and upload your
solution to gradescope by running it on the test data.

In [120]:
# Define a range of lambda values to test
lambda_values = [0.01, 0.1, 1, 10, 100]
best_lambda = None
best_mse = float('inf')

# Function to train the bias model with a given lambda
def train_bias_model_with_lambda(user_ids, item_ids, ratings, lambda_reg, iterations=10, learning_rate=0.01):
    global alpha, user_bias, item_bias
    alpha = 1.0  # Reset global average
    user_bias = defaultdict(float)  # Reset user biases
    item_bias = defaultdict(float)  # Reset item biases

    for _ in range(iterations):
        for u, i, r in ratings:
            # Predicted rating based on current parameters
            pred = alpha + user_bias[u] + item_bias[i]
            
            # Calculate error
            error = r - pred  # Target is 1 for "read" status
            
            # Update parameters with regularization
            alpha += learning_rate * (error - lambda_reg * alpha)
            user_bias[u] += learning_rate * (error - lambda_reg * user_bias[u])
            item_bias[i] += learning_rate * (error - lambda_reg * item_bias[i])

# Train and evaluate the model for each lambda
for lambda_reg in lambda_values:
    # Train the model with the current lambda
    train_bias_model_with_lambda(train_user_ids, train_book_ids, ratingsTrain, lambda_reg)
    
    # Calculate MSE on the validation set
    mse = calculate_mse(ratingsValid)
    
    # Check if this is the best lambda so far
    if mse < best_mse:
        best_mse = mse
        best_lambda = lambda_reg

# Ensure lambda and MSE are stored as floats
lamb = float(best_lambda)
validMSE = float(best_mse)

In [121]:
answers['Q8'] = (lamb, validMSE)

In [122]:
assertFloat(answers['Q8'][0])
assertFloat(answers['Q8'][1])

In [123]:
lamb, validMSE

(0.01, 1.449611497627705)

In [124]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("assignment1/pairs_Rating.csv"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,b = l.strip().split(',')
  if u in userAverage:
    predictions.write(u + ',' + b + ',' + str(userAverage[u]) + '\n')
  else:
    predictions.write(u + ',' + b + ',' + str(globalAverage) + '\n')
    
predictions.close()

In [125]:
f = open("answers_hw1.txt", 'w') # Write your answers to a file
f.write(str(answers) + '\n')
f.close()